In [2]:
import numpy as np
import pandas as pd

## OLMAR

On-Line Portfolio Selection with Moving Average Reversion.

Reference:
B. Li, S. C.H. Hoi.
https://icml.cc/2012/papers/168.pdf

In [3]:
data=pd.read_csv('./Data_OLPS/sp500.csv',encoding="gb18030")
print('data shape：',data.shape)

data shape： (1276, 25)


In [4]:
#data.info()
N=data.shape[0]
d=data.shape[1]
x=np.zeros((N,d))
b=np.ones(d)/d
x=data.to_numpy()
x

array([[1.01773646, 0.99428331, 1.00952381, ..., 0.97113071, 1.03318242,
        1.04351768],
       [0.98671935, 1.00574956, 1.00628931, ..., 0.97440132, 1.00145751,
        0.98262381],
       [1.00841313, 0.98808463, 0.996875  , ..., 1.03559322, 1.02478367,
        0.98054819],
       ...,
       [0.99481641, 1.02232692, 0.99833956, ..., 1.03733122, 1.03882726,
        1.00492287],
       [0.97915762, 0.96653977, 0.98898129, ..., 0.98315467, 0.96058988,
        0.9694644 ],
       [1.02616408, 0.98383085, 1.00483498, ..., 1.00311526, 1.02038115,
        1.01465387]])

In [5]:
def simplex_proj(y):
    """ Projection of y onto simplex. """
    m = len(y)
    bget = False

    s = sorted(y, reverse=True)
    tmpsum = 0.

    for ii in range(m-1):
        tmpsum = tmpsum + s[ii]
        tmax = (tmpsum - 1) / (ii + 1);
        if tmax >= s[ii+1]:
            bget = True
            break

    if not bget:
        tmax = (tmpsum + s[m-1] -1)/m

    return np.maximum(y-tmax,0.)


### Algorithm 1. Portfolio Selection with OLMAR.
1. Input: $𝜖>1$: Reversion threshold; $w>3$: Window size; $x_1^n$: Market sequence.
2. Output: $S_n$: Cumulative wealth after $n^{th}$ periods.
3. Procedure:
4. Initialize $𝒃_𝟏 =(\frac{1}{𝑚},⋯,\frac{1}{𝑚}),S_0 = 1$ 
5. for t = 1, 2,...,n do
6. $\quad$Receive stock price relatives: $𝑥_𝑡=(𝑥_{t1},⋯,𝑥_{tm})$
7. $\quad$Calculate daily return and cumulative return: $S_t=S_{t-1}×(b_t·x_𝑡)$
8. $\quad$Predict next price relative vector: 
$$
x̃_{𝑡+1}= \frac{1}{w} (1+\frac{1}{x_𝑡}+ ⋯ + \frac{1}{⊗_{i=0}^{w-2}\quad x_{t-i}})
$$

9. $\quad$Update the portfolio:

$$
b_{t+1}=OLMAR(ϵ, w, x_{t+1}, b_t)
$$

In [6]:
#  Simple moving average, p totals have omega-1 row data,its ,relative_prices
def SMA_predict(omega,p,d): # omega: SMA Window size , d = p.shape[1] -> stocks_number
    unit_vector = np.ones(d)
    SMA_predict = np.zeros(d)
    SMA_predict += unit_vector
    for i in range(1,omega):
        x = np.ones(d)
        for j in range(i):
            x *= p.iloc[omega-j-2,:]
        SMA_predict += 1.0/x
    SMA_predict = SMA_predict/omega
    return SMA_predict


### Algorithm 2.   OLMAR($ϵ, w, x_{t+1}, b_t$)
1. Input: $𝜖>1$: Reversion threshold; $w>3$: Window size; $x̃_{𝑡+1}$: Predicted price relatives; $b_t$:  Current portfolio.
2. Output: $b_{t+1}$: Next portfolio.
3. Procedure:
4. Calculate the following variables:
$$\bar x_{𝑡+1}=\frac{1^T x̃_{𝑡+1}}{m},λ_{t+1}=max⁡(0,\frac{ε-b_t ⋅ x̃_{𝑡+1})}{‖x̃_{𝑡+1}−\bar x_{𝑡+1}‖^2})$$
5. Update the portfolio:
$$b_{t+1}=b_t+λ_{t+1}(x̃_{𝑡+1}−\bar x_{𝑡+1})$$
6. Normalize portfolio: $$𝒃_{𝑡+1}=argmin‖𝒃−𝒃_𝑡 ‖^2$$

In [7]:
cum_ret=1 # Initialization
epsilon=5 # Reversion threshold
w=10 # SMA Window size
daily_r=np.ones(N)
b=np.ones(d)/d

for t in range(N-1):
    x[t+1] = SMA_predict(w,data,d)
    x_bar = np.mean(x[t+1])
    denominator = np.sum((x[t+1] - np.mean(x[t+1]))**2)
    molecular  = epsilon - np.dot(b,x[t+1])
    lambd = np.maximum(0, molecular / denominator)
    b = b + lambd * (x[t+1] - x_bar) #- gamma * (y[t+1] - y_bar)
    b = simplex_proj(b)
    daily_r[t] = np.dot(x[t+1],b)
    cum_ret = cum_ret * daily_r[t] 
print("Cumulative return:",cum_ret,"-----> Expressed by scientific counting:",f'{cum_ret:1.2e}') 


Cumulative return: 56122.95391049031 -----> Expressed by scientific counting: 5.61e+04
